# Simulating graded electrodes

In this notebook we explore how to simulate the effect of graded electrodes in the performance of a battery. Graded electrodes have a composition that varies along the thickness of the electrode, typically active material volume fraction and particle size. This variation can be used to improve the performance of the battery, for example, by increasing the power density.

As usual, we start by importing PyBaMM.

In [1]:
%pip install "pybamm[plot,cite]" -q # install PyBaMM if it is not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


We use the DFN model for the simulations and the Chen2020 parameter set. Note that we will need to modify the default Chen2020 parameter set to describe graded electrodes.

In [2]:
model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues("Chen2020")

We will vary the porosity in both electrodes and we will try three different scenarios: constant porosity, one where lower porosity occurs near the separator and one where lower porosity occurs near the current collector. All other parameters are kept constant. The varying porosity is defined to be linear centered around the default value and with a variation of $\pm$ 10%.

We define the varying porosities and store them in a list so we can loop over when solving the model.

In [3]:
L_n = parameter_values["Negative electrode thickness [m]"]
L_s = parameter_values["Separator thickness [m]"]
L_p = parameter_values["Positive electrode thickness [m]"]

eps_n_0 = parameter_values["Negative electrode porosity"]
eps_p_0 = parameter_values["Positive electrode porosity"]

eps_ns = [
    eps_n_0,
    lambda x: eps_n_0 * (1.1 - 0.2 * (x / L_n)),
    lambda x: eps_n_0 * (0.9 + 0.2 * (x / L_n)),
]

eps_ps = [
    eps_p_0,
    lambda x: eps_p_0 * (0.9 - 0.2 / L_p * (L_n + L_s) + 0.2 * (x / L_p)),
    lambda x: eps_p_0 * (1.1 + 0.2 / L_p * (L_n + L_s) - 0.2 * (x / L_p)),
]

Note that the distance through the electrode is computed from the negative electrode, so parameter need to be defined accordingly. Next, we can just solve the models for the various parameter sets. We apply a fairly high C-rate to see the effect of the graded electrodes on the discharge capacity.

In [4]:
solutions = []

experiment = pybamm.Experiment(["Discharge at 3C until 2.5 V"])

for eps_n, eps_p in zip(eps_ns, eps_ps):
    parameter_values["Negative electrode porosity"] = eps_n
    parameter_values["Positive electrode porosity"] = eps_p
    sim = pybamm.Simulation(
        model,
        parameter_values=parameter_values,
        experiment=experiment,
        solver=pybamm.IDAKLUSolver(),
    )
    sol = sim.solve()
    solutions.append(sol)

And plot the results:

In [5]:
pybamm.dynamic_plot(
    solutions,
    labels=[
        "Constant porosity",
        "Low porosity at separator",
        "High porosity at separator",
    ],
)

interactive(children=(FloatSlider(value=0.0, description='t', max=673.9136958613059, step=6.7391369586130585),…

We observe that, even though the average porosity is the same for the three cases the discharge capacity is much higher with the graded electrode where porosity is higher near the separator. This is because the higher porosity near the separator facilitates the ion transport and the better utilisation of the active material.

As a sanity check we can plot the porosity profiles for the three cases and see they match what we intended.

In [8]:
pybamm.dynamic_plot(
    solutions,
    output_variables=["Negative electrode porosity", "Positive electrode porosity"],
    labels=[
        "Constant porosity",
        "Low porosity at separator",
        "High porosity at separator",
    ],
)

interactive(children=(FloatSlider(value=0.0, description='t', max=673.9136958613059, step=6.7391369586130585),…

In [9]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Von DAG Bruggeman. Berechnung verschiedener physikalischer konstanten von heterogenen substanzen. i. dielektrizitätskonstanten und leitfähigkeiten der mischkörper aus isotropen substanzen. Annalen der physik, 416(7):636–664, 1935.
[3] Chang-Hui Chen, Ferran Brosa Planella, Kieran O'Regan, Dominika Gastol, W. Dhammika Widanage, and Emma Kendrick. Development of Experimental Techniques for Parameterization of Multi-scale Lithium-ion Battery Models. Journal of The Electrochemical Society, 167(8):080534, 2020. doi:10.1149/1945-7111/ab9050.
[4] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. 